# Project: Insights into the TMDb movie data

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

This data set contains information
about 10,000 movies collected from
The Movie Database (TMDb),
including user ratings and revenue.
>● Certain columns, like ‘cast’
and ‘genres’, contain multiple
values separated by pipe (|)
characters.

>● There are some odd characters
in the ‘cast’ column. Don’t worry
about cleaning them. You can
leave them as is.

>● The final two columns ending
with “_adj” show the budget and
revenue of the associated movie
in terms of 2010 dollars,
accounting for inflation over
time

In [1]:
# Use this cell to set up import statements for all of the packages that you
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from pandas.plotting import scatter_matrix


# Remember to include a 'magic word' so that your visualizations are plotted
#   inline with the notebook. See this page for more:
#   http://ipython.readthedocs.io/en/stable/interactive/magics.html


<a id='wrangling'></a>
## Data Wrangling

### General Properties

In [2]:
# load dataset
df = pd.read_csv('tmdb-movies.csv')
df.head()
#df.tail()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/15,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/15,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/15,2947,7.3,2015,1.747999e+08,1.385749e+09


In [3]:
# check datatypes
#df.info();
df.shape

(10866, 21)

In [4]:
# datatype checklist
#id                      10866 non-null int64 ok
#imdb_id                 10856 non-null object ok
#popularity              10866 non-null float64 ok
#budget                  10866 non-null int64 not ok --> convert to float
#revenue                 10866 non-null int64 not ok --> convert to float
#original_title          10866 non-null object ok
#cast                    10790 non-null object ok --> maybe seperate
#homepage                2936 non-null object ok
#director                10822 non-null object ok
#tagline                 8042 non-null object ok
#keywords                9373 non-null object ok --> maybe seperate
#overview                10862 non-null object ok --> maybe too long for analysis
#runtime                 10866 non-null int64 ok --> prob. in minutes
#genres                  10843 non-null object not ok --> seperate
#production_companies    9836 non-null object not ok --> seperate
#release_date            10866 non-null object not ok --> convert to datetime 
#vote_count              10866 non-null int64 ok
#vote_average            10866 non-null float64 ok
#release_year            10866 non-null int64 ok
#budget_adj              10866 non-null float64 ok
#revenue_adj             10866 non-null float64 ok

In [5]:
# check for duplicates
df.duplicated().sum()

1

In [6]:
# check for nulls
df.isnull().sum()

id                         0
imdb_id                   10
popularity                 0
budget                     0
revenue                    0
original_title             0
cast                      76
homepage                7930
director                  44
tagline                 2824
keywords                1493
overview                   4
runtime                    0
genres                    23
production_companies    1030
release_date               0
vote_count                 0
vote_average               0
release_year               0
budget_adj                 0
revenue_adj                0
dtype: int64

**Columns not needed for analysis**: 
>  - *homepage*, no additional informatin
>  - *Keywords* since too fuzzy
>  - *overview* since too fuzzy
>  - *imdb_id*  since no API to imdb is intended

**Columns where null rows does NOT need to be cleaned**: 
>  - *Tagline* is acceptable since not every movie has a Tagline

**Columns where null rows need to be cleaned**: 
>  - *cast*, *director* is acceptable due to low samplesize
>  - *genres* is relevant for further analysis
>  - *director* is relevant for further analysis

Conclusion: Work with two different dataframes for now. **df** and **df_full** and drop not needed columns.

### Data Cleaning
> After discussing the structure of the data and any problems that need to be
>   cleaned, perform those cleaning steps in the second part of this section.


In [7]:
# drop duplicates
df.drop_duplicates(inplace=True)

In [8]:
# drop not needed columns
df.drop(['homepage', 'keywords', 'overview', 'imdb_id'], axis=1, inplace=True)

In [9]:
## convert to float
df['revenue'] = df['revenue'].astype(float)
df['budget']  = df['budget'].astype(float)

In [10]:
# drop rows with null entries
df_full = df.copy()
df.dropna(subset=['cast', 'genres', 'director'], inplace=True)
df.isnull().sum()

# reset index to enable usage of iloc
df.reset_index(inplace=True)
df.index

RangeIndex(start=0, stop=10731, step=1)

**Prepare cast column for analysis**: 
> create different columns for the first 5 named actors in string



In [11]:
# check if each entry contains seperator
df_cast = df[df['cast'].str.contains('|')]
df.equals(df_cast)

True

In [12]:
# columns to split by "|"
split_columns = ['cast0', 'cast1', 'cast2', 'cast3', 'cast4', 'cast5']


# append empty cast columns
df['amnt_cast'] = ""
for c in split_columns:
    df[c] = ""
    
# get amount of actors
df['amnt_cast'] = df['cast'].apply(lambda x: len(x.split("|")))

# create subset for each amnt of actors
df1 = df.query('amnt_cast == 1')
df2 = df.query('amnt_cast == 2')
df3 = df.query('amnt_cast == 3')
df4 = df.query('amnt_cast == 4')
df5 = df.query('amnt_cast == 5')

# write actors from string to columns
# 1st actor
tmp = df1['cast'].apply(lambda x: x.split("|")[0])
df.iloc[tmp.index, df.columns.get_loc('cast0')] = tmp
# 2nd actor
for idx,val in enumerate(split_columns[0:2]):
    tmp = df2['cast'].apply(lambda x: x.split("|")[idx])
    df.iloc[tmp.index, df.columns.get_loc(val)] = tmp
# 3rd actor
for idx,val in enumerate(split_columns[0:3]):
    tmp = df3['cast'].apply(lambda x: x.split("|")[idx])
    df.iloc[tmp.index, df.columns.get_loc(val)] = tmp
# 4th actor
for idx,val in enumerate(split_columns[0:4]):
    tmp = df4['cast'].apply(lambda x: x.split("|")[idx])
    df.iloc[tmp.index, df.columns.get_loc(val)] = tmp
# 5th actor
for idx,val in enumerate(split_columns[0:5]):
    tmp = df5['cast'].apply(lambda x: x.split("|")[idx])
    df.iloc[tmp.index, df.columns.get_loc(val)] = tmp
df.drop(columns='cast5', inplace=True)

In [13]:
# double check
df.query('amnt_cast==1')
df.query('amnt_cast==2')
df.query('amnt_cast==3')
df.query('amnt_cast==4')
df.query('amnt_cast==5')

# drop original cast column
df.drop(columns='cast', inplace=True)

**Prepare genres column for analysis**: 
> create different columns for the first 5 named genres in string


In [14]:
# columns to split by "|"
split_columns = ['genre0', 'genre1', 'genre2', 'genre3', 'genre4', 'genre5']


# append empty cast columns
df['amnt_genre'] = ""
for c in split_columns:
    df[c] = ""
    
# get amount of actors
df['amnt_genre'] = df['genres'].apply(lambda x: len(x.split("|")))
max(df['amnt_genre'])

# create subset for each amnt of actors
df1 = df.query('amnt_genre == 1')
df2 = df.query('amnt_genre == 2')
df3 = df.query('amnt_genre == 3')
df4 = df.query('amnt_genre == 4')
df5 = df.query('amnt_genre == 5')

# write actors from string to columns
# 1st actor
tmp = df1['genres'].apply(lambda x: x.split("|")[0])
df.iloc[tmp.index, df.columns.get_loc('cast0')] = tmp
# 2nd actor
for idx,val in enumerate(split_columns[0:2]):
    tmp = df2['genres'].apply(lambda x: x.split("|")[idx])
    df.iloc[tmp.index, df.columns.get_loc(val)] = tmp
# 3rd actor
for idx,val in enumerate(split_columns[0:3]):
    tmp = df3['genres'].apply(lambda x: x.split("|")[idx])
    df.iloc[tmp.index, df.columns.get_loc(val)] = tmp
# 4th actor
for idx,val in enumerate(split_columns[0:4]):
    tmp = df4['genres'].apply(lambda x: x.split("|")[idx])
    df.iloc[tmp.index, df.columns.get_loc(val)] = tmp
# 5th actor
for idx,val in enumerate(split_columns[0:5]):
    tmp = df5['genres'].apply(lambda x: x.split("|")[idx])
    df.iloc[tmp.index, df.columns.get_loc(val)] = tmp
df.drop(columns='genre5', inplace=True)

In [15]:
# double check
df.query('amnt_genre==1')
df.query('amnt_genre==2')
df.query('amnt_genre==3')
df.query('amnt_genre==4')
df.query('amnt_genre==5')

# drop original cast column
df.drop(columns='genres', inplace=True)

In [16]:
# convert release date to datetime
df["release_date"]= pd.to_datetime(df["release_date"]) 

<a id='eda'></a>
## Exploratory Data Analysis


### Research Question 1
#### Which genres are most popular from year to year? Polour with regard to the TMDb Score and the resulting "chart" list.

In [17]:
# find unique genres
column_list = ['genre0', 'genre1', 'genre2', 'genre3', 'genre4']
df[column_list].describe()

,genre0,genre1,genre2,genre3,genre4
count,10731,10731,10731,10731,10731
unique,21,21,21,21,20
top,,,,,
freq,2271,2271,5680,8759,10192


In [41]:
# get overview over all subgernes
genres = df.groupby(column_list).popularity.mean()

# append all genres and its corresponding pobularity
df_genres = pd.DataFrame([], columns = ['genres', 'popularity', 'release_year'])

# append all subgenres to one genre list
for val in column_list:
    tmp = df[[val, 'popularity', 'release_year']]
    tmp.rename(columns={val: "genres"}, inplace=True)
    df_genres = pd.concat([df_genres,tmp])

# group subgenres
genres = df_genres.groupby('genres').popularity.mean()
genres

# store top genres for later analysis
nmax = 5
top_genres = genres.sort_values(ascending=False)
top_genres = top_genres.index[0:nmax].tolist()


In [42]:
# get most and least favourite genre
most_pop  = genres.idxmax()
idx0 = np.where(genres.index == most_pop)[0]
idx0 = idx0[0]

least_pop = genres.idxmin()
idx1 = np.where(genres.index == least_pop)[0]
idx1 = idx1[0]

In [43]:
df_genres

,genres,popularity,release_year
0,Chris Pratt,32.985763,2015
1,Tom Hardy,28.419936,2015
2,Shailene Woodley,13.112507,2015
3,Harrison Ford,11.173104,2015
4,Vin Diesel,9.335014,2015
...,...,...,...
10726,Chip Fitzwater,0.080598,1966
10727,Brian Bedford,0.065543,1966
10728,Lyubov Dobrzhanskaya,0.065141,1966
10729,Tadao Nakamaru,0.064317,1966


In [ ]:
# plot data 
fig, ax = plt.subplots(figsize=(25, 45))
ax.tick_params(axis='both', which='major', labelsize=25)

barlist = plt.barh(genres.index, genres)
plt.title('Pobularity across all genres', fontsize=30)
plt.xlabel('Genre', fontsize=30)
plt.ylabel('Pobularity according to TMDb', fontsize=30);

# change color for most popular genre
barlist[idx0].set_color('r')
barlist[idx1].set_color('g')


# legend
colors = {'Most Popular':'red', 'Least Popular':'green'}         
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
plt.legend(handles, labels, fontsize=30)

In [ ]:
# get unique release_years
unique_yrs = df_genres.release_year.unique()
unique_yrs = sorted(unique_yrs)

#genre_dict = {}
#genre_mat = []
# get unique genres
unique_genres = df_genres.genres.unique()

# pre-allocate dataframe
df_yr = pd.DataFrame(np.zeros((len(unique_genres), len(unique_yrs))), columns=unique_yrs)
df_yr.index = unique_genres
df_yr

# loop through unique years and determine ranking of most popular genres
for idx, val in enumerate(unique_yrs):
        # get genre popularity for each year
        df_single_yr = df_genres.query('release_year == %s' % val)
        genres = df_single_yr.groupby('genres').popularity.mean()
        
        # assign top 10
        n_max = 16
        genres.sort_values(ascending=False,inplace=True)
        for i in range(n_max):
            df_yr.loc[genres.index[i], val] = n_max - i
        #genre_dict.update({val:genres})
        #genre_mat = np.append(genre_mat, genres,1)
df_yr

In [ ]:
# drop 0 rows
df_yr = df_yr.replace(0, np.nan)
df_yr = df_yr.dropna(how='all', axis=0)

In [ ]:
# transpose for plotting
df_yr = df_yr.transpose()

> In order to provide a certain clarity anlysis focuses on the **Top 5 genres**.
> Which are the following:
> 1. Adventure
> 2. Science Fiction
> 3. Fantasy
> 4. Action
> 5. Animation

In [ ]:
# generate list of top geners to be displayed
df_plot = df_yr[top_genres]
df_plot.plot.area(figsize=(15,10))
#df_plot


##### Make visualization more clear
> Visualization struggles so running mean should be applied to the score
> 

In [ ]:
df_plot = df_plot.replace(np.nan, 0)
df_plot_mean = df_plot.rolling(window=10).mean()
#df_plot_mean.plot.area(figsize=(15,10),stacked=False)
#df_plot_mean.plot.line(figsize=(15,10))

# plot data 
fig, ax = plt.subplots(figsize=(35, 20))
marker_list = ['*-', '+-', 'o-', '.-', '^-']
ax.tick_params(axis='both', which='major', labelsize=25)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, top_genres)

# loop top genres
#linelist = np.zeros(np.shape(top_genres))
linelist = []
for idx, val in enumerate(top_genres):
    tmp, = plt.plot(df_plot_mean[val].index, df_plot_mean[val], marker_list[idx], markersize=14)
    linelist.append(tmp)
    
    
# change ylabel since we want to display a Rank and not a score

#plt.ylim([15, 0])
plt.title('Pobularity across Top 5 overall genres over the years', fontsize=30)
plt.xlabel('year', fontsize=30)
plt.ylabel('Chart Position according to TMDb Popularity', fontsize=30);
plt.legend(linelist, top_genres, fontsize=30)

locs, labels = plt.yticks() 
labels[2]
plt.yticks(np.linspace(0,15, num=6), np.round(np.linspace(16,1, num=6),decimals=0));

# change color for most popular genre
#barlist[idx0].set_color('r')
#barlist[idx1].set_color('g')


# legend
#colors = {'Most Popular':'red', 'Least Popular':'green'}         
#labels = list(colors.keys())
#handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
#plt.legend(handles, labels, fontsize=30)

#### Detailed Chart for Comparison
> in order to be able to obtain specific non filtered ranking of the genres

> In particular its good to observe that the upper ranks are almost always filled by the Top5 Genres, which means that we didnt miss any genre, which only was strong at a certain period of time.

In [ ]:
df_plot.plot.line(figsize=(30,10), style="o", markersize=15)


### Research Question 2
#### What kinds of properties are associated with movies that have high revenues?

In [ ]:
# get rough overview with regard to revenue column
scatter_matrix(df, alpha=0.2, figsize=(20, 20), diagonal='kde');


##### Overview to identify numerical attributes which have some kind of correlation with the revenue
> Numerical attributes to be considered:
> - budget (pretty clustered but existent)
> - popularity (pretty clustered but existent)
> - release_yr (much more revenue on the later years)
> - runtime (strong but check normal distribution)

> String attributes to be considered later on:
> - actors

In [ ]:
# deeper look at budget correlation
df.plot(x="budget_adj", y="revenue_adj", kind="scatter");

In [ ]:
# look at the disribution of budget for all movies
df.plot(y="budget_adj", kind="hist");

In [ ]:
# deeper look at budget correlation
df.plot(x="popularity", y="revenue_adj", kind="scatter");

In [ ]:
# look at the disribution of budget for all movies
df.plot(y="revenue_adj", kind="hist");

In [ ]:
# deeper look at release_year correlation
df.plot(y="revenue_adj", x="release_year", kind="scatter");

In [ ]:
# look at the disribution of release_year for all movies
df.plot(y="release_year", kind="hist");

In [ ]:
# deeper look at runtime correlation
df.plot(y="revenue_adj", x="runtime", kind="scatter");

In [ ]:
# look at the disribution of runtime for all movies
df.plot(y="runtime", kind="hist");

In [ ]:
top_features = df.query('budget > budget.mean()')
top_features.describe()

In [ ]:
df.describe()

In [ ]:
# find actors with movies of high revenue
column_list = ['cast0', 'cast1', 'cast2', 'cast3', 'cast4']
actors = df.groupby(column_list).revenue.mean()

# append all genres and its corresponding revenue
df_actors = pd.DataFrame([], columns = ['actors', 'revenue_adj', 'popularity'])

# append all actors to one actors list
for val in column_list:
    tmp = df[[val, 'revenue_adj', 'popularity']]
    tmp.rename(columns={val: "actors"}, inplace=True)
    df_actors = pd.concat([df_actors,tmp])

# store top genres for later analysis
actors = df_actors.groupby('actors').revenue_adj.mean()
top_actors = actors.sort_values(ascending=False)
top_actors

<a id='conclusions'></a>
## Conclusions Question 1
#### Which genres are most popular from year to year? Polour with regard to the TMDb Score and the resulting "chart" list.

> In general one can state the following:
> - A lot of people love the **Adventure** Genre. This is the genre which received on average the most popularity.
> - In Compaison the **Documentary** Genre seems to be the least liked genre over the years

> When you look at the the trend over the last 50 years:
> - it becomes remarkable that **Fantasy** Genre received a stunning groth in popularity.
> - the **Animation** Genre struggled it in the mid 1980s, when it was strong before, but become very popular later on
> - the **Adventure** Genre stayed remain over the years very popular and had only a slight dip in arround the year 2000
> - The **Action** and the the **Sci-Fi** Genere could be considered as an honerable 2nd place. Both remained very famous over the years but struggle to become the overall audiance favourite. There is a strong (optical) correlation between these.
    

<a id='conclusions'></a>
## Conclusions Question 2
#### Continue to explore the data to address your additional research

> The most obvious attributes for a high revenue turned mostly out somewhat true
> - A bigger **budget** correlates in general with more revenue for a movie
> - A bigger **popularity** correlates in general with more revenue for a movie

>However the correlation is not very strong and a a big budget or persumably popular topic for your movie, will not nesscarily lead to high revenues.

> The less obvious parameter with some correlation with the revenue, can be explained by the distribution. Meaning vor example that much more moview were produced with a certain **runtime** and thus have a higher chance to land big revenues. Same applies to the **release year**. Just a lot more younger movies can be found in the DB.

> In Addition the **actors** have been identified, who were able to score the most average revenue per movies they participated in.
> 1. Daisy Ridley          1.902723e+09 USD
> 2. Robert MacNaughton    1.791694e+09 USD
> 3. Ben Wright            1.574815e+09 USD
> 4. Betty Lou Gerson      1.574815e+09 USD
> 5. Martha Wentworth      1.574815e+09 USD

> It is no guarentee to receive a high revenue when you hire these actors, but a pretty good start - provided they are still alive ;) 


## Concerns / Disclaimer
> - This analysis is limited to the TMDb. 
> - Popularity and even genres and somewhat subjektive criteria (it depends on the group of people who determines these attributes
> - The transformation of the popularity into a chart system/ranking, can not consider differences of the popularity with high or narrow gaps. Its just an order and serves to overlook the data in a more understandable way.
> - **Revenue** and **Budget** was used with its adjusted values (which takes inflation into consideration)
> - Second anlysis technically favour old movies, since they had more time to generate revenue.